In [1]:
import requests
import pandas as pd

df = pd.read_excel('../data samples/Indications (AMS)_Australia_19.xlsx')
df.head()

,Indication,User alias,Tags,My tags,Groups
0,Compound or open fracture,Compound or open fracture,"broken bone, Bone and Joint,trauma,wound",NaN,Bone and Joint
1,Infected prosthetic joint or bone prosthesis,Infected prosthetic joint or bone prosthesis,"Infected joint prosthesis, hip, knee, Bone and...",NaN,Bone and Joint
2,Osteomyelitis,Osteomyelitis,"OM, Pyogenic inflammation of bone, OSTM, Bone ...",NaN,Bone and Joint
3,Reactive arthritis (Reiter's disease),Reactive arthritis (Reiter's disease),Bone and Joint,NaN,Bone and Joint
4,Septic arthritis,Septic arthritis,"Bacterial arthritis, bacterial synovitis, sacr...",NaN,Bone and Joint


In [2]:
selected_df = df[['Indication', 'Tags','Groups']]
selected_df.head()

,Indication,Tags,Groups
0,Compound or open fracture,"broken bone, Bone and Joint,trauma,wound",Bone and Joint
1,Infected prosthetic joint or bone prosthesis,"Infected joint prosthesis, hip, knee, Bone and...",Bone and Joint
2,Osteomyelitis,"OM, Pyogenic inflammation of bone, OSTM, Bone ...",Bone and Joint
3,Reactive arthritis (Reiter's disease),Bone and Joint,Bone and Joint
4,Septic arthritis,"Bacterial arthritis, bacterial synovitis, sacr...",Bone and Joint


In [3]:
df['Groups'].unique(), df['Groups'].nunique()

(array(['Bone and Joint', 'Central Nervous System',
        'Cardiovascular system', 'Ear nose and throat', 'Eye',
        'Gastrointestinal', 'Systemic infections', 'Genital',
        'Intra-abdominal', 'Oral and Dental', 'Other',
        'Prophylaxis - medical', 'Respiratory', 'Sepsis and bacteraemia',
        'Skin and Soft Tissue', 'Prophylaxis - surgical', 'Urinary tract'],
       dtype=object),
 17)

In [4]:
CODE_SYSTEM_ID = '645a4f69203d1d8b3fbb80b4'
# requests create group
group_names = df['Groups'].unique()
group_ress = []
for gname in group_names:
    group_res = requests.post("http://localhost:8000/center/groups",json={
        "name": gname,
        "code_system_id": CODE_SYSTEM_ID
    }).json()
    if group_res['code']!=200:
        print(gname)
        print(group_res)
        print()
    group_ress.append(group_res)

Bone and Joint
{'code': 201, 'data': {'id': '6464eecc4b12e2ba809f8859', 'name': 'Bone and Joint'}, 'msg': 'ok'}

Central Nervous System
{'code': 201, 'data': {'id': '6464eecd4b12e2ba809f885b', 'name': 'Central Nervous System'}, 'msg': 'ok'}

Cardiovascular system
{'code': 201, 'data': {'id': '6464eecd4b12e2ba809f885d', 'name': 'Cardiovascular system'}, 'msg': 'ok'}

Ear nose and throat
{'code': 201, 'data': {'id': '6464eecd4b12e2ba809f885f', 'name': 'Ear nose and throat'}, 'msg': 'ok'}

Eye
{'code': 201, 'data': {'id': '6464eecd4b12e2ba809f8861', 'name': 'Eye'}, 'msg': 'ok'}

Gastrointestinal
{'code': 201, 'data': {'id': '6464eecd4b12e2ba809f8863', 'name': 'Gastrointestinal'}, 'msg': 'ok'}

Systemic infections
{'code': 201, 'data': {'id': '6464eecd4b12e2ba809f8865', 'name': 'Systemic infections'}, 'msg': 'ok'}

Genital
{'code': 201, 'data': {'id': '6464eecd4b12e2ba809f8867', 'name': 'Genital'}, 'msg': 'ok'}

Intra-abdominal
{'code': 201, 'data': {'id': '6464eece4b12e2ba809f8869', 'name

In [5]:
for group_res in group_ress:
    print(group_res['data']['id'],group_res['data']['name'])
    

6464eecc4b12e2ba809f8859 Bone and Joint
6464eecd4b12e2ba809f885b Central Nervous System
6464eecd4b12e2ba809f885d Cardiovascular system
6464eecd4b12e2ba809f885f Ear nose and throat
6464eecd4b12e2ba809f8861 Eye
6464eecd4b12e2ba809f8863 Gastrointestinal
6464eecd4b12e2ba809f8865 Systemic infections
6464eecd4b12e2ba809f8867 Genital
6464eece4b12e2ba809f8869 Intra-abdominal
6464eece4b12e2ba809f886b Oral and Dental
6464eece4b12e2ba809f886d Other
6464eece4b12e2ba809f886f Prophylaxis - medical
6464eece4b12e2ba809f8871 Respiratory
6464eecf4b12e2ba809f8873 Sepsis and bacteraemia
6464eecf4b12e2ba809f8875 Skin and Soft Tissue
6464eecf4b12e2ba809f8877 Prophylaxis - surgical
6464eecf4b12e2ba809f8879 Urinary tract


In [6]:
df_id = pd.DataFrame({
    "GroupId": [res['data']['id'] for res in group_ress],
    "Groups": [res['data']['name'] for res in group_ress]
})
df_id

,GroupId,Groups
0,6464eecc4b12e2ba809f8859,Bone and Joint
1,6464eecd4b12e2ba809f885b,Central Nervous System
2,6464eecd4b12e2ba809f885d,Cardiovascular system
3,6464eecd4b12e2ba809f885f,Ear nose and throat
4,6464eecd4b12e2ba809f8861,Eye
5,6464eecd4b12e2ba809f8863,Gastrointestinal
6,6464eecd4b12e2ba809f8865,Systemic infections
7,6464eecd4b12e2ba809f8867,Genital
8,6464eece4b12e2ba809f8869,Intra-abdominal
9,6464eece4b12e2ba809f886b,Oral and Dental


In [7]:
groupid_merged_df = pd.merge(selected_df, df_id, left_on='Groups', right_on='Groups', how='left')
groupid_merged_df

,Indication,Tags,Groups,GroupId
0,Compound or open fracture,"broken bone, Bone and Joint,trauma,wound",Bone and Joint,6464eecc4b12e2ba809f8859
1,Infected prosthetic joint or bone prosthesis,"Infected joint prosthesis, hip, knee, Bone and...",Bone and Joint,6464eecc4b12e2ba809f8859
2,Osteomyelitis,"OM, Pyogenic inflammation of bone, OSTM, Bone ...",Bone and Joint,6464eecc4b12e2ba809f8859
3,Reactive arthritis (Reiter's disease),Bone and Joint,Bone and Joint,6464eecc4b12e2ba809f8859
4,Septic arthritis,"Bacterial arthritis, bacterial synovitis, sacr...",Bone and Joint,6464eecc4b12e2ba809f8859
...,...,...,...,...
599,Uncomplicated acute rhinosinusitis,"acute inflammation of sinus,acute inflammation...",Ear nose and throat,6464eecd4b12e2ba809f885f
600,"Sepsis, unknown source, community acquired, tr...","systemic infection,septic shock,empiric,septic...",Sepsis and bacteraemia,6464eecf4b12e2ba809f8873
601,"Invasive pulmonary fungal infection, other",NaN,Respiratory,6464eece4b12e2ba809f8871
602,"Fungal infection, Lomentospora",NaN,Systemic infections,6464eecd4b12e2ba809f8865


In [13]:
# requests create concept

concept_ress = []
for i, row in groupid_merged_df.iterrows():
    data = {
        'code_system_id': CODE_SYSTEM_ID,
        'group_id':row['GroupId'],
        'name':str(row['Indication']),
        'description':str(row['Tags'])
    }
    print(data)

    concept_res = requests.post("http://localhost:8000/center/concepts",json=data).json()
    print(concept_res)
    print()
    concept_ress.append(concept_res)

{'code_system_id': '645a4f69203d1d8b3fbb80b4', 'group_id': '6464eecc4b12e2ba809f8859', 'name': 'Compound or open fracture', 'description': 'broken bone, Bone and Joint,trauma,wound'}
{'code': 201, 'data': {'description': 'broken bone, Bone and Joint,trauma,wound', 'id': '6464eff983d44a482418cbd1', 'name': 'Compound or open fracture'}, 'msg': 'ok'}

{'code_system_id': '645a4f69203d1d8b3fbb80b4', 'group_id': '6464eecc4b12e2ba809f8859', 'name': 'Infected prosthetic joint or bone prosthesis', 'description': 'Infected joint prosthesis, hip, knee, Bone and Joint,arthroplasty device,PJI,replacement,pin-site,metal'}
{'code': 201, 'data': {'description': 'Infected joint prosthesis, hip, knee, Bone and Joint,arthroplasty device,PJI,replacement,pin-site,metal', 'id': '6464eff983d44a482418cbd3', 'name': 'Infected prosthetic joint or bone prosthesis'}, 'msg': 'ok'}

{'code_system_id': '645a4f69203d1d8b3fbb80b4', 'group_id': '6464eecc4b12e2ba809f8859', 'name': 'Osteomyelitis', 'description': 'OM, Py